In [26]:
## Load credentials from config/credentials.ini
#redo3.py
#Author: Graham Waters, 2023
#^ Essential Imports
import asyncio
import configparser
import logging
import os
import random
import sys
import time
import traceback
from datetime import datetime
#^ Non-Essential Imports
import alive_progress
import pandas as pd
import pandas_ta
import pandas_ta as ta
from alive_progress import alive_bar
from icecream import ic
from pytz import timezone
from robin_stocks import robinhood as r
from tqdm import tqdm
import numpy as np
#^ Constants
PCT_SPEND = 0.05 # 5% of buying power
verbose_mode = True # Set to True to see more logging output
from colorama import Back, Fore, Style
from ratelimit import limits, sleep_and_retry
config = configparser.ConfigParser()
config.read('config/credentials.ini')
# Define variables from config
# Define variables from config
coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
stop_loss_percent = float(config['trading']['stop_loss_percent'])
percent_to_use = float(config['trading']['percent_to_use'])
verbose_mode = config['logging'].getboolean('verbose_mode')
debug_verbose = config['logging'].getboolean('debug_verbose')
reset_positions = config['logging'].getboolean('reset_positions')
minimum_usd_per_position = float(config['trading']['minimum_usd_per_position'])
pct_to_buy_with = float(config['trading']['percent_to_use'])
pct_to_buy_per_trade = float(config['trading']['percent_to_spend_per_trade'])
username = config['robinhood']['username']
password = config['robinhood']['password']
login = r.login(username, password)
print(f"Logged in as {username}")
# Function to get current price for a coin


Logged in as "graham.waters37@gmail.com"


In [27]:
#!important
# Locked Funds
# Some coins should be locked from sale until they are redeemable.
locked_positions = {}

In [28]:
!pip install schedule

# MicroStrategy One: The Night Watchman
- Every night at Midnight CST, purchase $1.00 worth of Aave (AAVE) save the volume of coins purchased as a variable "v_night".
- Every Moring at 3:00 AM CST, sell exactly "v_night" worth of AAVE.


In [29]:
import schedule
import time
import configparser
from datetime import datetime

# Load credentials from config/credentials.ini
config = configparser.ConfigParser()
config.read('config/credentials.ini')
# minimum_usd_per_trade = float(config['trading']['minimum_usd_per_trade'])
minimum_usd_per_trade = 3.00 # for testing purposes
# save the total money to a file accessible by multiple scripts
with open('data/total_money.txt', 'r') as f:
    total_money = float(f.read())

# Define variables from config
coins = [coin.strip() for coin in config['trading']['coins'].split(', ')]
v_night = None

print(f'I can trade {coins} with a minimum of ${minimum_usd_per_trade} per trade.')


# Function to get current price for a coin
def get_current_price(coin_name):
    quote = r.crypto.get_crypto_quote(coin_name)
    return float(quote['mark_price'])

# Function to buy a coin
@sleep_and_retry
def buy_coin(coin_name, amount):
    try:
        current_price = get_current_price(coin_name)
        amount_to_buy = float(amount) / float(current_price)
        buying_usd = round(float(amount_to_buy), 2)
        buy_cost = round(float(buying_usd) * float(current_price), 2)
        result = r.orders.order_crypto(
            symbol=str(coin_name).upper(),
            amountIn='dollars',
            side='buy',
            quantityOrPrice=float(buy_cost),
            limitPrice=round(float(current_price),2),
            timeInForce='gtc',
            jsonify=True
        )
        # save result to a file
        with open('data/buy_orders.txt', 'a') as f:
            f.write(f'{result}\n')
        return result
    except Exception as e:
        logging.error(e)
        error = f'Error buying {coin_name} at {datetime.now()}'
        # save error to a file
        with open('data/errors.txt', 'a') as f:
            f.write(f'{error}\n')
        return error
# Function to sell a coin
@sleep_and_retry
def sell_coin(coin_name):
    try:
        current_price = get_current_price(coin_name)
        holdings = r.crypto.get_crypto_positions(info=None)
        coin_holdings = [coin for coin in holdings if coin['currency']['code'] == coin_name]
        amount_to_sell = float(coin_holdings[0]['quantity'])
        result = r.orders.order_crypto(
            symbol=coin_name,
            amountIn='quantity',
            side='sell',
            quantityOrPrice=float(amount_to_sell),
            limitPrice=round(float(current_price), 2),
            timeInForce='gtc',
            jsonify=True
        )
        return result
    except Exception as e:
        logging.error(e)
        error = f'Error selling {coin_name} at {datetime.now()}'
        # save error to a file
        with open('data/errors.txt', 'a') as f:
            f.write(f'{error}\n')
        return error
coins_dict = {}

# Function to execute the strategy
def execute_strategy():
    
    global v_night, total_money
    # default value for v_night will be 0.00 as we don't have any coins to sell at the start
    v_night = 0.00 if v_night is not None else v_night
    for coin in tqdm(coins, desc='Coins', leave=False, total=len(coins)):
        print(f'Executing strategy for {coin}')
        # Buy $1.00 worth of each coin at midnight
        quantity_purchased = 1.00 / get_current_price(coin)
        schedule.every().day.at("00:00").do(buy_coin, coin, minimum_usd_per_trade)
        total_money -= 1.00
        # Save the number of coins purchased at midnight
        coins_dict[coin] = quantity_purchased

        # Sell the purchased coins at 3:00 AM
        schedule.every().day.at("03:00").do(sell_coin, coin_name=coin)

        # Save the total money to a file accessible by multiple scripts
        with open('data/total_money.txt', 'w') as f:
            f.write(str(total_money))
    # Keep the script running
    while True:
        schedule.run_pending()
        time.sleep(1)

# Run the strategy
execute_strategy()


Coins:   0%|          | 0/14 [00:00<?, ?it/s]

I can trade ['BTC', 'ETH', 'DOGE', 'SHIB', 'ETC', 'UNI', 'AAVE', 'LTC', 'LINK', 'COMP', 'AVAX', 'XLM', 'BCH', 'XTZ'] with a minimum of $3.0 per trade.
Executing strategy for BTC


Coins:   7%|▋         | 1/14 [00:00<00:04,  3.24it/s]

Executing strategy for ETH


Coins:  14%|█▍        | 2/14 [00:00<00:03,  3.42it/s]

...

Coins:  79%|███████▊  | 11/14 [00:02<00:00,  4.20it/s]

Executing strategy for XLM


Coins:  93%|█████████▎| 13/14 [00:03<00:00,  4.42it/s]

Executing strategy for BCH
Executing strategy for XTZ


TypeError: sell_coin() takes 1 positional argument but 2 were given